In [ ]:
pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.8 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import pandas as pd
import numpy as np
import ast
import re
import sklearn
import six
from abc import ABCMeta
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from scipy import sparse
import warnings
warnings.filterwarnings('ignore')
import nltk
import six
import re
import gensim
from tqdm import tqdm


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from abc import ABCMeta
from sklearn.pipeline import Pipeline
from sklearn.utils import check_X_y, check_array
from scipy import sparse
from nltk.stem import PorterStemmer,WordNetLemmatizer
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.utils import resample
from sklearn.metrics import accuracy_score,f1_score
from gensim.models import FastText

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tsel = pd.read_csv(r'/content/drive/MyDrive/TA/Prepro_Pelayanan.csv', usecols=['Username','Text','Pelayanan','text_result'])
tsel

,Username,Text,Pelayanan,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,"['handphone', 'lapor', 'wajib', 'bantu', 'laca..."
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",0.0,"['tidak tahu', 'telkomsel', 'lambat', 'kuota',..."
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,0.0,"['kendala', 'sinyal', 'telkomsel']"
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,-1.0,"['tidak bisa', 'nomor', 'mati', 'dihidupakan',..."
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",-1.0,"['admin', 'kemarin', 'aplikasi', 'punya', 'tel..."
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",0.0,"['sinyal', 'minus', 'tolong', 'baik', 'problem..."
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,0.0,"['punya', 'telkomsel', 'ganggu', 'aduh', 'desa..."
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,"['allah', 'tambah', 'kerja', 'esuk', 'neng', '..."
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,0.0,"['admin', 'sinyal', 'telkomsel', 'rumah', 'dae..."


In [ ]:
tsel['text_result'] = tsel['text_result'].apply(str)

In [ ]:
import ast

def join_text_list(text):
    texts = ast.literal_eval(text)
    return ' '.join([text for text in texts])

tsel['text_result'] = tsel['text_result'].apply(join_text_list)
tsel

,Username,Text,Pelayanan,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,handphone lapor wajib bantu lacak imei perihal...
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",0.0,tidak tahu telkomsel lambat kuota promo murah ...
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,0.0,kendala sinyal telkomsel
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,-1.0,tidak bisa nomor mati dihidupakan grapari nomo...
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",-1.0,admin kemarin aplikasi punya telkomsel ganggu ...
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",0.0,sinyal minus tolong baik problem location cama...
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,0.0,punya telkomsel ganggu aduh desak pakai
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,allah tambah kerja esuk neng grapari
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,0.0,admin sinyal telkomsel rumah daerah rawabunga ...


## Ektraksi Fitur TF-IDF 

In [ ]:
vectors= TfidfVectorizer(ngram_range=(1,2), max_features = 1000)
vectors

TfidfVectorizer(max_features=1000, ngram_range=(1, 2))

In [ ]:
tfidf = vectors.fit_transform(tsel['text_result'].astype(str))
tfidf

<16991x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 116944 stored elements in Compressed Sparse Row format>

In [ ]:
df_tfidf = tfidf.copy()

In [ ]:
X = df_tfidf
y = tsel['Pelayanan']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=5)

### AdaBoost


In [ ]:
pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
adBoost = AdaBoostClassifier()

## Ekspansi Fitur

In [ ]:
# Load Model trained
model_exp = FastText.load('/content/drive/MyDrive/TA/model_ntwt/fasttext/Twitter_Berita.fasttext').wv  
#Penggunaan Corpus disesuaikan (Corpus Berita, Twitter, dan Twitter+Berita ) 

In [ ]:
def feature_expansion(df,feature):
    for col in tqdm(df.columns): #loop per kolom
        try: 
            sim_word = model_exp.similar_by_word(col, topn = 20) #Mencari similarity berdasarkan nilai n
        except:
            sim_word = []
        if sim_word != []: #jika similarity tidak kosong
            for term in [sim_word[i][0] for i in range(len(sim_word))]: #loop per-word yang ada di Similarity
                if term in feature:
                    #untuk semua kolom yang mempunyai nilai 0 di kolom, tetapi mempunyai nilai yang bukan 0 pada kolom term
                    #nilainya diganti dengan nilai kolom term yang mempunyai nilai bukan 0
                    df[col][(df[col]==0) & (df[term]!=0)] = df[term][(df[col]==0) & (df[term]!=0)]
    return df

In [ ]:
feature_ef = vectors.get_feature_names()

In [ ]:
# Membuat dataframe data test & train
tsel_x_train = pd.DataFrame(x_train.todense(), columns = feature_ef)
tsel_x_test = pd.DataFrame(x_test.todense(), columns = feature_ef)

In [ ]:
dfexp_x_train= feature_expansion(tsel_x_train, feature_ef)

100%|██████████| 1000/1000 [00:21<00:00, 46.21it/s]


In [ ]:
dfexp_x_test = feature_expansion(tsel_x_test, feature_ef)

100%|██████████| 1000/1000 [00:25<00:00, 39.98it/s]


## Klasifikasi

In [ ]:
y_pred_exp = adBoost.fit(dfexp_x_train, y_train).predict(dfexp_x_test)
y_true = y_test

### Top 1 

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[ 104  135    6]
 [  38 1265    8]
 [  23   95   26]]
              precision    recall  f1-score   support

        -1.0       0.63      0.42      0.51       245
         0.0       0.85      0.96      0.90      1311
         1.0       0.65      0.18      0.28       144

    accuracy                           0.82      1700
   macro avg       0.71      0.52      0.56      1700
weighted avg       0.80      0.82      0.79      1700

Precision:  0.7088189588189588
Recall:  0.5233192107252258
F1:  0.563855037695067
Accuracy:  0.8205882352941176


In [ ]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.6558688313316028

### Top 5

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[ 105  134    6]
 [  38 1265    8]
 [  23   95   26]]
              precision    recall  f1-score   support

        -1.0       0.63      0.43      0.51       245
         0.0       0.85      0.96      0.90      1311
         1.0       0.65      0.18      0.28       144

    accuracy                           0.82      1700
   macro avg       0.71      0.52      0.57      1700
weighted avg       0.80      0.82      0.79      1700

Precision:  0.7097501115573405
Recall:  0.5246797549429129
F1:  0.5651727950251294
Accuracy:  0.8211764705882353


In [ ]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.6546293606902512

### Top 10

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[ 105  131    9]
 [  32 1266   13]
 [  16   91   37]]
              precision    recall  f1-score   support

        -1.0       0.69      0.43      0.53       245
         0.0       0.85      0.97      0.90      1311
         1.0       0.63      0.26      0.36       144

    accuracy                           0.83      1700
   macro avg       0.72      0.55      0.60      1700
weighted avg       0.81      0.83      0.80      1700

Precision:  0.7213998684948738
Recall:  0.5503969767413703
F1:  0.5989263331707898
Accuracy:  0.8282352941176471


In [ ]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.6722280557696904

### Top 20

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[  99  133   13]
 [  39 1261   11]
 [  21   94   29]]
              precision    recall  f1-score   support

        -1.0       0.62      0.40      0.49       245
         0.0       0.85      0.96      0.90      1311
         1.0       0.55      0.20      0.29       144

    accuracy                           0.82      1700
   macro avg       0.67      0.52      0.56      1700
weighted avg       0.79      0.82      0.79      1700

Precision:  0.6724191857712856
Recall:  0.5224438987392567
F1:  0.5618504459572132
Accuracy:  0.8170588235294117


In [ ]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.6545204767417783